In [209]:
# ✅ 1. Install required libraries
!pip install --quiet langchain langchain-openai faiss-cpu python-dotenv pymupdf openai

In [210]:
# ✅ 2. Import necessary packages
import os
import re
from dotenv import load_dotenv
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI


In [211]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

embeddings = OpenAIEmbeddings(openai_api_key=api_key)
llm = OpenAI(openai_api_key=api_key)


In [212]:
#pdf loader

In [213]:
from langchain.document_loaders import PyMuPDFLoader

In [214]:
# ✅ Load PDF document
pdf_path = "/Users/jessicahong/Desktop/textclustering.pdf"
loader = PyMuPDFLoader(pdf_path)

In [215]:
# ✅ Load the documents as a list of LangChain Document objects
documents = loader.load()

In [216]:
# ✅ Preview the first 100 characters of the first page
print(documents[0].page_content[:100])

The peer-reviewed version of this paper is published in the International Journal of Cognitive Compu


In [217]:
#Text Splitting

In [218]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [219]:
# ✅ Initialize the text splitter
# chunk_size: Maximum number of characters per chunk
# chunk_overlap: Number of overlapping characters between chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

In [220]:
# ✅ Split the loaded documents into smaller chunks
# Input should be a list of Document objects
texts = text_splitter.split_documents(documents)

In [221]:
# ✅ Print how many chunks were created
print(f"Number of text chunks: {len(texts)}")

Number of text chunks: 71


In [222]:
#Embeddings

In [223]:
#RAG Chain creation

In [224]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [225]:
vectorstore = FAISS.from_documents(texts, embeddings)

In [226]:
# 5. Build RetrievalQA chain using OpenAI LLM and FAISS retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=api_key),  
    retriever=vectorstore.as_retriever()
)

In [227]:
# 6. Run your query against the RAG system
query = "What are the main contributions of this paper?"
result = qa_chain.run(query)
print(result)

 The main contributions of this paper are testing and identifying optimal combinations of embeddings and clustering algorithms, evaluating the effectiveness of LLM embeddings compared to traditional techniques, and assessing the impact of model size and summarization techniques on clustering performance. 


In [228]:
#Text cleaning

In [229]:
import re

In [230]:
#clean_text : cleans input text by removing page numbers, collapsing extra blank lines, and stripping unwanted line characters
def clean_text(text):
    # Remove page numbers, e.g., "Page 1 of 10"
    text = re.sub(r"Page \d+ of \d+", "", text)
    # Replace multiple newlines with a single newline
    text = re.sub(r"\n\s*\n", "\n", text)
    # Remove special characters except word characters, whitespace, and basic punctuation
    text = re.sub(r"[^\w\s.,?!]", "", text)
    # Trim leading and trailing whitespace
    return text.strip()

In [231]:
documents = loader.load()

In [232]:
#loops through each document object in the documents list
for doc in documents:
    # Clean the text content of each document using the clean_text function
    doc.page_content = clean_text(doc.page_content)
    #applies clean_text func to the page_content attribute of each document to remove unwanted characters, numbers, and extra blank lines

In [233]:
# Text splitting
# Splits long texts into smaller chunks, each about 1000 characters long
# chunk_overlap=100 means each chunk overlaps the previous chunk by 100 characters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)  # Split the cleaned documents into smaller chunks


In [234]:
#FAISS VECTORSTORE

In [235]:
# Save the FAISS vector store to local disk
vectorstore.save_local("faiss_index")

In [236]:
#Later, load from disk
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

In [237]:
# ✅ Set allow_dangerous_deserialization=True ONLY IF you trust the data source
vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

In [240]:
# Recreate embedding object with your OpenAI API key
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [241]:
#RAG

In [242]:
# Initialize OpenAI LLM with your API key
llm = OpenAI(openai_api_key=api_key)

In [243]:
# Create the RetrievalQA chain with the vectorstore retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever()
)


In [245]:
# Run a query against your PDF documents
query = "What are the main contributions of this paper?"
result = qa_chain.run(query)

In [246]:
print(result)

 The main contribution of this paper is the testing and identifying of optimal combinations of embeddings and clustering algorithms for text clustering tasks. The paper also discusses advancements in text embeddings and their impact on clustering performance. Additionally, the paper explores the use of LLM embeddings and their superiority over traditional embedding techniques. The results of the study and recommendations for future developments are also presented. 


In [247]:
#FAISS Index optimization => Create IVF(Inverted File index)

In [248]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [249]:
vectorstore = FAISS.from_documents(texts, embeddings)

In [250]:
import faiss
import numpy as np

In [251]:
# the length of the embedding vector
dimension = len(embeddings.embed_query("test"))

In [252]:
print(dimension)

1536


In [253]:
#IVF(Inverted File index)
"""
What is IVF : used to speed up vector similarity search by partitioning the entire vector space into multiple clusters (also called centroids), and assigning each vector to its nearest cluster.
why use IVF : Searching through all vectors in a large dataset is computationally expensive and slow. IVF helps by limiting the search to only the most relevant clusters, dramatically improving search speed.
How it work? 
The vector space is first partitioned into n clusters using k-means clustering. This step requires calling index.train(vectors).
Each vector is assigned to its closest centroid (cluster center).
During search, instead of scanning all clusters, only a few relevant clusters are searched, making the process much faster.
"""

'\nWhat is IVF : used to speed up vector similarity search by partitioning the entire vector space into multiple clusters (also called centroids), and assigning each vector to its nearest cluster.\nwhy use IVF : Searching through all vectors in a large dataset is computationally expensive and slow. IVF helps by limiting the search to only the most relevant clusters, dramatically improving search speed.\nHow it work? \nThe vector space is first partitioned into n clusters using k-means clustering. This step requires calling index.train(vectors).\nEach vector is assigned to its closest centroid (cluster center).\nDuring search, instead of scanning all clusters, only a few relevant clusters are searched, making the process much faster.\n'

In [254]:
# IVF index setup
nlist = 10  # num of cluster
quantizer = faiss.IndexFlatL2(dimension)  # # Flat index for L2 distance (used as the coarse quantizer)


In [255]:
# Create an IVF flat index (coarse quantizer + flat index within each cluster)
# - quantizer: used to assign vectors to clusters
# - dimension: dimensionality of the embedding vectors
# - nlist: number of clusters (coarse centroids)
# - faiss.METRIC_L2: use L2 (Euclidean) distance for similarity search
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

In [256]:
# Retrieve vectors from the existing FAISS index inside the vectorstore
# reconstruct_n(start, count) returns 'count' vectors starting from index 'start'
vectors = vectorstore.index.reconstruct_n(0, vectorstore.index.ntotal)  # (ntotal, dimension) ndarray

In [257]:
# debugging
print(type(vectors), vectors.shape)

<class 'numpy.ndarray'> (70, 1536)


In [258]:
 # vectors가 ndarray임을 확인했으면 별도 변환 불필요
# 만약 리스트라면 numpy array로 변환
if not isinstance(vectors, np.ndarray):
    vectors = np.array(vectors).astype('float32')

In [259]:
# 인덱스 학습 확인 후 학습
if not index.is_trained:
    print("Index is not trained. Training now...")
    index.train(vectors)   # 인덱스가 학습되지 않았으면 벡터로 학습함
index.add(vectors)        # 학습된 인덱스에 벡터 추가
print("Vectors added successfully.")

Index is not trained. Training now...
Vectors added successfully.


WARNING clustering 70 points to 10 centroids: please provide at least 390 training points
